In [1]:
from google.colab import files

uploaded = files.upload()

Saving document.txt to document.txt
Saving V3.csv to V3.csv


In [2]:
!pip install llama-index==0.10.9
!pip install llama-index-llms-huggingface
!pip install pymongo
!pip install sentence_transformers
!pip install langchain
!pip install llama-index-embeddings-langchain
!pip install --upgrade nbconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cache

In [3]:
from llama_index.core import VectorStoreIndex,ServiceContext,Document, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from llama_index.core import PromptTemplate
from pymongo import MongoClient
import pandas as pd
import numpy as np
import csv
import os

In [4]:
# Access the secret directly from Colab
import os

# Retrieve the Hugging Face token from the Colab secret
secret_value = os.getenv("HF_TOKEN")

# Log in to Hugging Face using the retrieved token
!huggingface-cli login --token $secret_value --add-to-git-credential


Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    service.run()
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/user.py", line 98, in run
    login(token=self.args.token, add_to_git_credential=self.args.add_to_git_credential)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py", line 109, in login
    _login(token, add_to_git_credential=add_to_git_credential, write_permission=write_permission)
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_login.py", line 305, in _login
    raise ValueError("Invalid token passed!")
ValueError: Invalid token passed!


In [5]:
csv_file ="V3.csv"

In [6]:
df = pd.read_csv(csv_file)
df = df.iloc[:,1:]

In [7]:
df

,Year,Locality,Estimated Value,Sale Price,Property,Residential,num_rooms,num_bathrooms,carpet_area,property_tax_rate,Face
0,2009,Waterbury,111440.0,185000.0,Single Family,Detached House,3,3,996.0,1.025953,South
1,2009,NaN,73080.0,152000.0,Single Family,Detached House,3,1,935.0,1.025953,North
2,2009,Waterbury,50540.0,105000.0,Single Family,Detached House,3,1,951.0,1.025953,East
3,2009,Norwalk,571060.0,1000000.0,Single Family,Detached House,3,2,NaN,1.025953,South
4,2009,Bridgeport,4775276.0,272900.0,Single Family,Detached House,3,1,971.0,1.025953,East
...,...,...,...,...,...,...,...,...,...,...,...
9995,2022,NaN,155550.0,215000.0,Three Family,Triplex,6,5,2114.0,1.003979,North
9996,2022,Greenwich,NaN,2162500.0,Single Family,Detached House,3,3,933.0,1.003979,North
9997,2022,West Hartford,124740.0,225635.0,Single Family,Detached House,3,1,922.0,1.003979,North
9998,2022,NaN,1455230.0,2625000.0,Single Family,Detached House,3,2,977.0,1.003979,North


In [8]:
output_dir = 'C:/Users/bagch/Downloads/real estate dataset'    # Directory to save the text files

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for i, row in enumerate(reader):
        text = row['Property']  # Replace 'text_column' with the name of the column containing the text
        text_file = os.path.join(output_dir, f'document_{i}.txt')
        with open(text_file, 'w') as f:
            f.write(text)

In [9]:
reader = SimpleDirectoryReader(input_files=["document.txt"])
data = reader.load_data()

from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

embed_model = LangchainEmbedding(HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
system_prompt="""
Welcome to the Smart Real Estate Chatbot!

Please adhere to the following guidelines:
- Ask only real estate-related questions.
- Avoid using inappropriate language.
- Refrain from engaging in harmful activities.

If your request does not meet these criteria, the chatbot will respond with "I am not supposed to reply to such queries".

Your objective is to provide accurate house information and address inquiries within the real estate domain.
"""

## Default format for Llama2
query_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [11]:
import torch

llm= HuggingFaceLLM(
context_window=4096,
max_new_tokens=256,
generate_kwargs={"temperature":0.0, "do_sample": False},
system_prompt=system_prompt,
query_wrapper_prompt=query_prompt,
tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
model_name="meta-llama/Llama-2-7b-chat-hf",
device_map="auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
sevice_context=ServiceContext.from_defaults(chunk_size=1024, llm=llm,embed_model=embed_model)

<ipython-input-12-ee7913f65658>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sevice_context=ServiceContext.from_defaults(chunk_size=1024, llm=llm,embed_model=embed_model)


In [13]:
index = VectorStoreIndex.from_documents(data,service_context=sevice_context)

In [14]:
index

In [15]:
query_engine = index.as_query_engine()

In [16]:
query_engine.query("Hey who you are?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Response(response="Hello! I'm just an AI assistant, here to help you with any questions or concerns you may have about the property. Is there anything specific you would like to know or discuss?", source_nodes=[NodeWithScore(node=TextNode(id_='28f636f5-ca41-47eb-a4f0-c99ed6699736', embedding=None, metadata={'file_path': 'document.txt', 'file_name': 'document.txt', 'file_type': 'text/plain', 'file_size': 400, 'creation_date': '2024-04-18', 'last_modified_date': '2024-04-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ccc69230-ee09-45e9-ace9-d64814b4061a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'document.txt', 'file_name': 'document.txt', 'file_type': 'text/plain', 'file_size': 400

In [18]:
query_engine.query("Find me a room of single family")

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 443.06 MiB is free. Process 4230 has 14.31 GiB memory in use. Of the allocated memory 13.92 GiB is allocated by PyTorch, and 271.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)